In [1]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
#!pip uninstall tensorflow

In [ ]:
#!pip install tensorflow==1.15

In [2]:
import tensorflow as tf
import tensorboard as tb

In [3]:
from keras import backend as K

Using TensorFlow backend.


In [4]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models,layers, Input, optimizers, initializers, regularizers, metrics
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator ,array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard,  EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten

In [5]:
import os
from glob import glob
import PIL
from PIL import Image

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import keras
import cv2

from os import listdir
from os.path import isfile, join
import random
import shutil

In [ ]:
#!pip uninstall keras

In [ ]:
#!pip install keras==2.3.1

In [6]:
print(tf.__version__)

1.15.0


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [7]:
os.getcwd()

'/home/caitech/Desktop/yujung'

In [ ]:
#os.chdir('/content/drive/My Drive/posters')

In [ ]:
# !mkdir -p '/content/drive/My Drive/posters'

In [8]:
#test셋과 train셋으로 데이터 나눔

renew_database = 1
#파일 처리 - error예방
if(renew_database):
    if os.path.exists('train'):
        shutil.rmtree('train')
    if os.path.exists('test'):
        shutil.rmtree('test')
    if not os.path.exists('train'):
        os.mkdir('train')
    if not os.path.exists('test'):
        os.mkdir('test')



    rate=0.1 #분할 비율
    
    for root_dir in ['0', '1']:
        list_file = os.listdir(root_dir)
        picknumber=int(len(list_file)*rate) #test이미지 개수
        test_pic = random.sample(list_file, picknumber) #test중애서 랜덤으로 뽑기
        os.mkdir('train/'+root_dir)
        os.mkdir('test/'+root_dir)
        for item in list_file:
            if item in test_pic:
                shutil.copy(root_dir+'/'+item,'test/'+root_dir+'/'+item)
            else:  #나머지는 train
                shutil.copy(root_dir+'/'+item,'train/'+root_dir+'/'+item)

In [9]:
train_dir = os.path.join('/home/caitech/Desktop/yujung/train/')
test_dir = os.path.join('/home/caitech/Desktop/yujung/test/')

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
IMAGE_SIZE = 224

In [ ]:
#데이터를 읽고 이미지를 한번에 넣은 개수_train
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE),batch_size = 40,class_mode='binary')

In [ ]:
#데이터를 읽고 이미지를 한번에 넣은 개수_validation
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE),batch_size = 10, class_mode='binary')

In [ ]:
print(len(os.listdir(train_dir+'./1_down')))
print(len(os.listdir(train_dir+'/0')))

In [ ]:
math.ceil(train_generator.n / train_generator.batch_size)

In [ ]:
math.ceil(train_generator.n / train_generator.batch_size)

In [ ]:
#vgg model생성
pre_trained_vgg = VGG16(weights = 'imagenet',include_top = False,input_shape = (224,224,3))

In [ ]:
#pre_trained_vgg.summary()

In [ ]:
for layer in pre_trained_vgg.layers:
    layer.trainable = False

In [ ]:
vgg_model = tf.keras.Sequential()
vgg_model.add(pre_trained_vgg)
vgg_model.add(layers.Flatten())
vgg_model.add(layers.Dense(1024,activation = 'relu'))
vgg_model.add(Dropout(0.5))
vgg_model.add(layers.Dense(512,activation = 'relu'))
vgg_model.add(Dropout(0.5))
vgg_model.add(layers.Dense(256,activation = 'relu'))
vgg_model.add(Dropout(0.5))
vgg_model.add(layers.Dense(2,activation = 'softmax'))

vgg_model.summary()

In [ ]:
early_stopping = EarlyStopping(patience = 30)

In [ ]:
vgg_model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
history_vgg = vgg_model.fit_generator(
        train_generator,
        steps_per_epoch = math.ceil(train_generator.n / train_generator.batch_size),
        epochs=100,
        validation_data=test_generator,
        validation_steps = math.ceil(test_generator.n / test_generator.batch_size),
        callbacks = [early_stopping]
        )

In [ ]:
scores = vgg_model.evaluate_generator(test_generator, steps=math.ceil(test_generator.n / test_generator.batch_size))

In [ ]:
scores

In [ ]:
import pandas as pd
history_vgg_df = pd.DataFrame(history_vgg.history)
scores_df = pd.DataFrame(scores)

In [ ]:
history_vgg_df.to_excel('./down_batch40_10_e100/vgg_history.xlsx')
scores_df.to_excel('./down_batch40_10_e100/vgg_scores.xlsx')

In [ ]:
#!nvidia-smi

In [ ]:
#Confution Matrix and Classification Report
#Y_pred = model.predict_generator(validation_generator, num_of_test_samples // batch_size+1)
Y_pred = vgg_model.predict_generator(test_generator, math.ceil(test_generator.n / test_generator.batch_size))
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
len(y_pred)

In [ ]:
#y_pred

In [ ]:
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')

In [ ]:
#target_names = ['Cats', 'Dogs', 'Horse']
target_names = ['0', '1']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

In [ ]:
vgg_report = classification_report(test_generator.classes, y_pred, target_names=target_names,output_dict=True)

In [ ]:
vgg_report_df = pd.DataFrame(vgg_report).transpose()

In [ ]:
vgg_report_df.to_excel('./down_batch40_10_e100/vgg_report.xlsx')

In [ ]:
epochs = range(len(history_vgg.history['acc']))

plt.plot(epochs,history_vgg.history['acc'], label='Training acc')
plt.plot(epochs, history_vgg.history['val_acc'],label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

In [ ]:
#plt.figure()
plt.plot(epochs,history_vgg.history['loss'],label='Training loss')
plt.plot(epochs,history_vgg.history['val_loss'], label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

#plt.show()

Inception-V3

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import RMSprop

In [ ]:
#크기조절
train_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
IMAGE_SIZE = 299

In [ ]:
#데이터를 읽고 이미지를 한번에 넣은 개수_train
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=40,
                                                        class_mode='binary')

In [ ]:
test_generator = test_datagen.flow_from_directory(test_dir, target_size = (IMAGE_SIZE,IMAGE_SIZE),batch_size = 10,
                                                 class_mode = 'binary')

In [ ]:
pre_inceptionV3 = InceptionV3(weights = 'imagenet',include_top=False,input_shape = (299,299,3))

In [ ]:
# pre_inceptionV3.summary()

In [ ]:
v3_model = tf.keras.Sequential()
v3_model.add(pre_inceptionV3)
v3_model.add(layers.GlobalAveragePooling2D())
v3_model.add(layers.Dense(2,activation = 'softmax'))

In [ ]:
#v3_model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop

In [ ]:
for layer in pre_inceptionV3.layers:
    layer.trainable = False

In [ ]:
early_stopping = EarlyStopping(patience =30)

In [ ]:
v3_model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam',
              #optimizer = 'RMSprop',
              metrics=['accuracy'],
             )

In [ ]:
history_inceptionv3= v3_model.fit_generator(
        train_generator,
        steps_per_epoch=23,
        epochs=100,
        validation_data=test_generator,
        validation_steps= 10,
        callbacks = [early_stopping]
        )

In [ ]:
v3_scores = v3_model.evaluate_generator(test_generator, steps=10)

In [ ]:
v3_scores

In [ ]:
v3_history_df = pd.DataFrame(history_inceptionv3.history)
v3_scores = pd.DataFrame(v3_scores)

In [ ]:
v3_history_df.to_excel('./down_batch40_10_e100/inceptionV3_history.xlsx')
v3_scores.to_excel('./down_batch40_10_e100/inceptionV3_scores.xlsx')

In [ ]:
epochs = range(len(history_inceptionv3.history['acc']))
plt.plot(epochs, history_inceptionv3.history['acc'],label = 'Training accuracy')
plt.plot(epochs,history_inceptionv3.history['val_acc'],label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

In [ ]:
plt.plot(epochs, history_inceptionv3.history['loss'],label='Training loss')
plt.plot(epochs,history_inceptionv3.history['val_loss'],label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

In [ ]:
Y_pred_v3 = v3_model.predict_generator(test_generator, 10)
y_pred_v3 = np.argmax(Y_pred_v3, axis=1)

In [ ]:
len(y_pred_v3)

In [ ]:
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred_v3))
print('Classification Report')

In [ ]:
#target_names = ['Cats', 'Dogs', 'Horse']
target_names = ['0', '1']
print(classification_report(test_generator.classes, y_pred_v3, target_names=target_names))

In [ ]:
v3_report = classification_report(test_generator.classes, y_pred_v3, target_names=target_names,output_dict=True)

In [ ]:
v3_report_df = pd.DataFrame(v3_report).transpose()

In [ ]:
v3_report_df.to_excel('./down_batch40_10_e100/inceptionV3_report.xlsx')

ResNet50

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
IMAGE_SIZE = 224

In [ ]:
#데이터를 읽고 이미지를 한번에 넣은 개수_train
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=40,
                                                        class_mode='binary')

In [ ]:
test_generator = test_datagen.flow_from_directory(test_dir, target_size = (IMAGE_SIZE,IMAGE_SIZE),batch_size = 10,
                                                 class_mode = 'binary')

In [ ]:
pre_resnet = ResNet50(weights = 'imagenet', include_top = False, input_shape = (224,224,3))

In [ ]:
for layer in pre_resnet.layers:
    layer.trainable = False

In [ ]:
# pre_resnet.summary()

In [ ]:
r_model = tf.keras.Sequential()
r_model.add(pre_resnet)
r_model.add(layers.GlobalAveragePooling2D())
r_model.add(layers.Dense(2,activation = 'softmax'))

In [ ]:
# r_model.summary()

In [ ]:
early_stopping = EarlyStopping(patience = 30)

In [ ]:
r_model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam',
              #optimizer = 'RMSprop',
              metrics=['accuracy'],
             )

In [ ]:
history_resnet = r_model.fit_generator(train_generator,
                                       steps_per_epoch=23,
                                       epochs=100,
                                       validation_data=test_generator,
                                       validation_steps= 10,
                                      callbacks = [early_stopping]
                                       )

In [ ]:
scores_resnet = r_model.evaluate_generator(test_generator,steps = 10)

In [ ]:
scores_resnet

In [ ]:
history_resnet_df = pd.DataFrame(history_resnet.history)
scores_resnet_df = pd.DataFrame(scores_resnet)

In [ ]:
history_resnet_df.to_excel('./down_batch40_10_e100/resnet50_history.xlsx')
scores_resnet_df.to_excel('./down_batch40_10_e100/resnet50_scores.xlsx')

In [ ]:
epochs = range(len(history_resnet.history['acc']))
plt.plot(epochs, history_resnet.history['acc'],label = 'Training accuracy')
plt.plot(epochs,history_resnet.history['val_acc'],label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

In [ ]:
plt.plot(epochs, history_resnet.history['loss'],label='Training loss')
plt.plot(epochs,history_resnet.history['val_loss'],label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

In [ ]:
Y_pred_resnet = r_model.predict_generator(test_generator, 10)
y_pred_resnet = np.argmax(Y_pred_resnet, axis=1)

In [ ]:
y_pred_resnet

In [ ]:
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred_resnet))
print('Classification Report')

In [ ]:
target_names = ['0', '1']
print(classification_report(test_generator.classes, y_pred_resnet, target_names=target_names))

In [ ]:
resnet_report = classification_report(test_generator.classes, y_pred_resnet, target_names=target_names,output_dict = True)

In [ ]:
resnet_report_df = pd.DataFrame(resnet_report).transpose()

In [ ]:
resnet_report_df.to_excel('./down_batch40_10_e100/resnet50_report.xlsx')

In [ ]:
with open('result.csv', 'a') as f:
    f.write('어떤결과')